# 🚀 Module 3: Model Packaging and Deployment on Kubernetes

In this module, we will:
1. Load the Trained Model from MLflow
2. Build a REST API for model inference
3. Create a Dockerfile to containerize the service
4. Deploy the container to a Kubernetes cluster (Minikube or OpenShift)
5. Optionally, expose and test the deployed service

## 🧳 Step 1: Load the Trained Model from MLflow  

Retrieve the registered model that you tained in the last step from the MLflow server and save it as a `.pkl` file for use in the API.


In [2]:
import mlflow
import joblib
import os

# Set the MLflow tracking URI
mlflow.set_tracking_uri("http://localhost:5000")

# Define model name and version or stage
model_name = "BikeSharingModel"
model_stage = "None"  # Change to "Production" or "Staging" if you're using stages

# Load the model from the MLflow Model Registry
model_uri = f"models:/{model_name}/{model_stage if model_stage != 'None' else '1'}"
model = mlflow.pyfunc.load_model(model_uri)

# Optionally, save it locally for container usage
os.makedirs("../models", exist_ok=True)
joblib.dump(model, "../models/bike_model.pkl")
print("Model downloaded from MLflow and saved to ../models/bike_model.pkl")


Model downloaded from MLflow and saved to ../models/bike_model.pkl


## 🛠️ Step 2: Create a REST API using FastAPI
This API will load the model and expose an endpoint for predictions.

In [3]:
%%writefile ../models/app.py
from fastapi import FastAPI
import joblib
import pandas as pd

app = FastAPI()
model = joblib.load("bike_model.pkl")

@app.post("/predict")
def predict(features: dict):
    df = pd.DataFrame([features])
    prediction = model.predict(df)[0]
    return {"prediction": prediction}

Writing ../models/app.py


## 📦 Step 3: Containerize with Docker
Create a Dockerfile for the FastAPI app.

In [4]:
%%writefile ../models/Cntainerfile
FROM python:3.9-slim
WORKDIR /app
COPY bike_model.pkl app.py ./
RUN pip install fastapi[all] joblib pandas
CMD ["uvicorn", "app:app", "--host", "0.0.0.0", "--port", "8000"]

Writing ../models/Dockerfile


## 🧱 Step 4: Build and Run Docker Container Locally

In [5]:
!docker build -t bike-api ../models
!docker run -d -p 8000:8000 --name bike-api bike-api

/usr/bin/sh: line 1: docker: command not found
/usr/bin/sh: line 1: docker: command not found


## ☸️ Step 5: Deploy to Kubernetes
Create a Kubernetes deployment and service manifest.

In [ ]:
%%writefile ../models/k8s_deployment.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: bike-api
spec:
  replicas: 1
  selector:
    matchLabels:
      app: bike-api
  template:
    metadata:
      labels:
        app: bike-api
    spec:
      containers:
      - name: bike-api
        image: bike-api:latest
        ports:
        - containerPort: 8000
---
apiVersion: v1
kind: Service
metadata:
  name: bike-api-service
spec:
  selector:
    app: bike-api
  ports:
    - protocol: TCP
      port: 80
      targetPort: 8000
  type: NodePort

## 🚀 Step 6: Deploy to Kubernetes

In [ ]:
!kubectl apply -f ../models/k8s_deployment.yaml

## 🧪 Step 7: Test the API Endpoint

In [ ]:
# Replace <NodePort> with the actual exposed port
!curl -X POST "http://localhost:<NodePort>/predict" -H "Content-Type: application/json" -d '{"temp": 25, "hum": 0.8, "windspeed": 0.1}'

## ✅ Summary
- Exported the trained model
- Built a FastAPI service for prediction
- Containerized the API using Docker
- Deployed the container to Kubernetes
- Exposed and tested the endpoint